# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [3]:
#This notebook will use the linearlearner alogorithim to create a benchmark model for the udacity capstone project. This 
#notebook will take a subset of the provided data and classify those that triggered the reward as those that did not. A 
#triggered reward is defined as a reward value of >0 from the value dictionary. The features that will be used for this 
#classification are age, income, and gender.


#benchmark process is modified form the Fraud Detection Case Study

#import key resources
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

#First the notebook will read in the three json files

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [4]:
#id is used in two of the file and the id in the profile dataframe will be renamed as 'customer_id'
profile.rename(columns={'id': 'customer_id'}, inplace=True)

In [5]:
# a complete dataframe merging the three files is constructed
complete_df = pd.concat([portfolio, profile, transcript], axis = 1)

In [6]:
#dummy variables are need for only gender in this classification model and it is then concated to the original dataframe
gender_dummy = pd.get_dummies(complete_df['gender'])
complete_df = pd.concat([complete_df, gender_dummy], axis = 1)

In [8]:
#next uneeded columns for this classifier are dropped
complete_df = complete_df.drop(['person', 'id', 'gender', 'became_member_on', 'offer_type', 'event', 'customer_id', 'reward', 
                               'channels', 'duration', 'difficulty', 'time'], axis = 1)

In [9]:
#inspect the dataframe head to ensure we have the correct columns
complete_df.head()

,age,income,value,F,M,O
0,118.0,NaN,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,0
1,55.0,112000.0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},1,0,0
2,118.0,NaN,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,0
3,75.0,100000.0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},1,0,0
4,118.0,NaN,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,0,0


In [10]:
#we need to unpack the value dictionary into its different columns and the concat that into the original dataframe and drop the
#value column
from pandas.io.json import json_normalize
df = json_normalize(complete_df['value'])

In [11]:
complete_df = pd.concat([complete_df, df], axis = 1)
complete_df = complete_df.drop(['value'], axis = 1) 

In [12]:
complete_df.head()

,age,income,F,M,O,amount,offer id,offer_id,reward
0,118.0,NaN,0,0,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,55.0,112000.0,1,0,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,118.0,NaN,0,0,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,75.0,100000.0,1,0,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,118.0,NaN,0,0,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [13]:
#remove duplicated reward column and inspect head
complete_df = complete_df.loc[:,~complete_df.columns.duplicated()]
complete_df.head()

,age,income,F,M,O,amount,offer id,offer_id,reward
0,118.0,NaN,0,0,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,55.0,112000.0,1,0,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,118.0,NaN,0,0,0,NaN,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,75.0,100000.0,1,0,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,118.0,NaN,0,0,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [14]:
#we need to remove value, offer_id, and amount
complete_df = complete_df.drop(['offer id', 'offer_id', 'amount'], axis = 1)

In [15]:
#inspect the head of the new dataframe and determine adjustments that need to be made in filtering the data
complete_df.head()

,age,income,F,M,O,reward
0,118.0,NaN,0,0,0,NaN
1,55.0,112000.0,1,0,0,NaN
2,118.0,NaN,0,0,0,NaN
3,75.0,100000.0,1,0,0,NaN
4,118.0,NaN,0,0,0,NaN


In [49]:
#if reward is NaN make 0
complete_df['reward'] = complete_df['reward'].fillna(0)

In [50]:
# drop NaNs from income , most correspond to an age of 118 which is the Age NaN
complete_df = complete_df.dropna()

In [51]:
#change rewards that are greater than 0 to a 1
complete_df.loc[complete_df['reward'] > 0] = 1

In [52]:
#description of cleaned dataframe
complete_df['reward'].value_counts()

0.0    14513
1.0      312
Name: reward, dtype: int64

In [53]:
#cleaned data fram
complete_df.head()

,age,income,F,M,O,reward
1,55.0,112000.0,1,0,0,0.0
3,75.0,100000.0,1,0,0,0.0
5,68.0,70000.0,0,1,0,0.0
8,65.0,53000.0,0,1,0,0.0
12,58.0,51000.0,0,1,0,0.0


In [21]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()


In [54]:
#function for splitting data into test and training
# split into train/test
def train_test_split(complete_df, train_frac= 0.8, seed=1):
    '''Shuffle the data and randomly split into train and test sets;
       separate the class labels (the column in transaction_df) from the features.
       :param df: Dataframe of all credit card transaction data
       :param train_frac: The decimal fraction of data that should be training data
       :param seed: Random seed for shuffling and reproducibility, default = 1
       :return: Two tuples (in order): (train_features, train_labels), (test_features, test_labels)
       '''
    
    # convert the df into a matrix for ease of splitting
    df_matrix = complete_df.as_matrix()
    
    # shuffle the data
    np.random.seed(seed)
    np.random.shuffle(df_matrix)
    
    # split the data
    train_size = int(df_matrix.shape[0] * train_frac)
    # features are all but last column
    train_features  = df_matrix[:train_size, :-1]
    # class labels *are* last column
    train_labels = df_matrix[:train_size, -1]
    # test data
    test_features = df_matrix[train_size:, :-1]
    test_labels = df_matrix[train_size:, -1]
    
    return (train_features, train_labels), (test_features, test_labels)



In [55]:
(train_features, train_labels), (test_features, test_labels) = train_test_split(complete_df, train_frac=0.8)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [61]:
# import LinearLearner
from sagemaker import LinearLearner

# specify an output path
prefix = 'creditcard'
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate LinearLearner
linear = LinearLearner(role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier',
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       epochs=15,
                       binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                       target_recall=0.9,
                       positive_example_weight_mult='balanced')                      

In [57]:
#pd.Series(train_labels, name = 'A').unique()

array([0., 1.])

In [59]:
# convert features/labels to numpy
train_x_np = train_features.astype('float32')
train_y_np = train_labels.astype('float32')

# create RecordSet
formatted_train_data = linear.record_set(train_x_np, labels=train_y_np)

In [62]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

2020-01-14 21:40:40 Starting - Starting the training job...
2020-01-14 21:40:42 Starting - Launching requested ML instances......
2020-01-14 21:41:49 Starting - Preparing the instances for training......
2020-01-14 21:43:04 Downloading - Downloading input data
2020-01-14 21:43:04 Training - Downloading the training image..Docker entrypoint called with argument(s): train
[01/14/2020 21:43:20 INFO 140021299226432] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'auto', u'lr_scheduler_step': u'auto', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'au


2020-01-14 21:43:35 Uploading - Uploading generated training model
2020-01-14 21:43:35 Completed - Training job completed
Training seconds: 37
Billable seconds: 37
CPU times: user 464 ms, sys: 17 ms, total: 481 ms
Wall time: 3min 11s


In [63]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')


-------------------------------------------------------------------------------------------------------------------------------!CPU times: user 649 ms, sys: 24.3 ms, total: 674 ms
Wall time: 10min 42s


In [64]:
# function from Fraud Detection Case Study
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


In [65]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True) # verbose means we'll print out the metrics

Metrics for simple, LinearLearner.

prediction (col)   0.0  1.0
actual (row)               
0.0               2901    2
1.0                  0   62

Recall:     1.000
Precision:  0.969
Accuracy:   0.999



In [66]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [67]:
delete_endpoint(linear_predictor)

Deleted linear-learner-2020-01-14-21-40-40-710
